## Prepare training data

In [ ]:
# %%capture cap

%load_ext autoreload
%autoreload 2


import os

os.environ['OMP_NUM_THREADS'] = "6,1"
os.environ["ASE_DFTB_COMMAND"] = "ulimit -s unlimited; /home/dlb/anaconda3/envs/dftb_plumed/bin/dftb+ > PREFIX.out"
os.environ["DFTB_PREFIX"] = "/home/dlb/Downloads/pbc-0-3"

# https://www.plumed.org/doc-v2.7/user-doc/html/_m_e_t_a_d.html
# https://github.com/dftbplus/dftbplus/issues/1064
# try with numpy 1.22　
# https://gitlab.com/Sucerquia/ase-plumed_tutorial

from ase.calculators.lj import LennardJones
# from ase.calculators.plumed import Plumed
from runner.plumed import Plumed
# from plumed import Plumed
from ase.constraints import FixedPlane
from ase.io import read
from ase import units

from ase.geometry.analysis import Analysis
from ase.constraints import FixAtoms, FixBondLengths

# import plumed

from ase.calculators.dftb import Dftb



ps = 1000 * units.fs

setup = [f"UNITS LENGTH=A TIME={1/ps} ENERGY={units.mol/units.kJ}",
        #  "d: DISTANCE ATOMS=1,2 ",
        "at8: FIXEDATOM AT=-0.3652684296711638,5.59639757164289,11.089411052917308",
        "at6: FIXEDATOM AT=-0.5388786945959876,6.471914514609469,10.07028636148657",
        "c1: CENTER ATOMS=at8,at6",
        "c2: CENTER ATOMS=67,69,73,77,79,83",
        "c3: CENTER ATOMS=7,9,11,13,15,17",
        "c4: CENTER ATOMS=79,83",
         "phi: TORSION ATOMS=c1,c2,c3,c4",
         "metad: METAD ARG=phi SIGMA=1.10 HEIGHT=15.0 BIASFACTOR=5 TEMP=300.0 PACE=1", #GRID_MIN=-pi GRID_MAX=pi GRID_BIN=1500
         # "ARG=d SIGMA=0.20,0.20 HEIGHT=1.20 BIASFACTOR=5 TEMP=300.0 PACE=500",
         # "GRID_MIN=0,0 GRID_MAX=1.0,1.0 GRID_BIN=150,150",
         "PRINT ARG=phi STRIDE=1 FILE=../results/test/plumed/COLVAR",
         # "CALC_RCT ",
         # "RCT_USTRIDE=10",
         # "...",
         "FLUSH STRIDE=1000"]


atoms = read('../data/structures/new_systems/ktu_002.cif')


# fix_atoms = FixAtoms(indices=[atom.index for atom in atoms if atom.symbol=='N' or atom.symbol=='Si' or atom.symbol=='O'])
# # # ch_bonds = Analysis(atoms).get_bonds("C", "H")[0]
# # # fix_bond_lengths = FixBondLengths(ch_bonds)
# # # atoms.set_constraint([fix_atoms, fix_bond_lengths])
# atoms.set_constraint(fix_atoms)

calc_base = Dftb(atoms=atoms,
            label='crystal',
            kpts=(1,1,1)
            )

# atoms.calc = Plumed(calc=calc_base,
#                     input=setup,
#                     timestep=timestep,
#                     atoms=atoms,
#                     kT=0.1,
#                     log='../results/test/plumed/plumed_log.log')

from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin

atoms.calc = calc_base

# dyn = Langevin(atoms, 0.1, temperature_K=0.1/units.kB, friction=0.1,
#                fixcm=True, trajectory='../results/test/machine_learning/dftb_500.xyz')
# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(atoms, dt=0.5*units.fs, trajectory='../results/test/machine_learning/dftb_500.traj')  
# https://wiki.fysik.dtu.dk/ase/ase/md.html

dyn.run(500)


In [ ]:
from ase.io import read, write
from ase.visualize import view

traj = read('../results/test/machine_learning/dftb_500.traj', index=":")
# traj = read('/home/dlb/Downloads/UnbiasMD.xyz', index=":")
# write("test/plumed/biased_MTD.xyz", traj, format="extxyz")
# traj_biased = read('test/plumed/biased_MTD.xyz', index=":")
# traj_unbiased = read('test/plumed/.xyz', index=":")

view(traj)
# print(traj[200].get_potential_energy())
# print(traj[200].get_forces()[33])

In [ ]:
# check if correct forces were written into traj file
# https://github.com/qsnake/ase/blob/master/ase/calculators/singlepoint.py
# atoms = read('../data/structures/new_systems/ktu_002.cif')

from ase.calculators.singlepoint import SinglePointCalculator

snap = traj[200].copy()

calc_base = Dftb(atoms=atoms,
            label='crystal',
            kpts=(1,1,1)
            )

snap.calc = calc_base

print(snap.get_potential_energy())
print(snap.get_forces()[33])

## Train GP model

In [ ]:
from ase.visualize import view
from ase import io

import numpy as np

traj_md = io.read("../results/test/machine_learning/dftb_500.traj", index=":")
energies_md = np.zeros(len(traj_md) )
forces_md = np.zeros( (len(traj_md), len(traj_md[0]), 3 ) )
for i, snap in enumerate(traj_md):
    energies_md[i] = snap.get_potential_energy()
    forces_md[i] = snap.get_forces()
# forces_md = np.load("../../data/xtb_md/forces_xtb_md.npy")
# energies_md = np.load("../../data/xtb_md/energies_xtb_md.npy")

# Training data:
# training_indices = np.sort(  np.random.choice(np.arange(0,10_000), 1000, replace=False) )
training_indices = np.sort(  np.arange(0, 500, 10) )  
traj_train = [traj_md[i] for i in training_indices]
energies_train = energies_md[training_indices]
forces_train = forces_md[training_indices]
train_data = {'trajectory': traj_train, 'energies': energies_train, 'forces': forces_train}

#Test data:
# test_indices = np.sort(  np.random.choice(np.arange(0,92795), 200, replace=False) ) 
test_indices = np.sort(  np.arange(100,200,2) ) 
traj_test = [traj_md[i] for i in test_indices]
energies_test = energies_md[test_indices]
forces_test = forces_md[test_indices]
test_data = {'trajectory': traj_test, 'energies': energies_test, 'forces': forces_test}

data_units = "ev_angstrom" # standard ase units

In [ ]:
# using with fande environment

import sys
sys.path.append("../../fande") # Adds higher directory to python modules path.
# sys.path.append("..") # Adds higher directory to python modules path.


hparams = {
    'dtype' : 'float64',
    'device' : 'gpu'
}

# Descriptors parameters:
soap_params = {
    'species': ["H", "C"],
    'periodic': True,
    'rcut': 6.0,
    'sigma': 0.5,
    'nmax': 5,
    'lmax': 5,
    'average': "off",
    'crossover': True,
    'dtype': "float64",
    'n_jobs': 10,
    'sparse': False,
    'positions': [7, 11, 15]
}



In [ ]:
%load_ext autoreload
%autoreload 2

from fande.data import FandeDataModuleASE
import matplotlib.pyplot as plt

fdm = FandeDataModuleASE(train_data, test_data, hparams, units=data_units)

print(fdm.forces_train_norm.shape)
plt.plot(fdm.forces_train[:,16,0])

In [ ]:

fdm.calculate_invariants(soap_params)
print(fdm.train_DX.shape)

In [ ]:
# https://gitlab.com/tsitsvero/nmr-knot-continue/-/blob/master/experimental.ipynb
import rascal.representations
from rascal.representations import SphericalInvariants

In [ ]:
%%time

import ase
from ase.io import read, write
from rascal.representations import SphericalInvariants

import rascal

N_dataset = 1000
# frames = read('/media/dlb/la_cie1/repos/librascal/examples/i-PI/zundel/zundel_dataset.xyz', index=':{}'.format(N_dataset))

# from ase.io import read, write
# from ase.visualize import view

traj = read('../results/test/machine_learning/dftb_500.traj', index=":")


frames = traj

for f in frames:
    f.wrap(eps=1e-18)

frame_test = frames[10]
# frame_test.cell = ase.cell.Cell([(10,0,0),(0,10,0),(0,0,10)])
# frame_test.cell = ase.cell.Cell([[100.,0.01,0.],[0.,100.,0.0001],[0.001,0.,100.]])
frame_test.pbc = True
frame_test.wrap(eps=1e-10)
# https://cosmo-epfl.github.io/librascal/_modules/rascal/neighbourlist/structure_manager.html#mask_center_atoms_by_id
# frame_test.arrays["center_atoms_mask"] = [False, False, True, False, False, False, False]

# rascal.neighbourlist.structure_manager.mask_center_atoms_by_id(frame_test, id_select=[0,3])
# rascal.neighbourlist.structure_manager.mask_center_atoms_by_species(frame_test, species_select=['C'])

# define the parameters of the spherical expansion
hypers = dict(soap_type="PowerSpectrum",
              interaction_cutoff=4.0,
              max_radial=5,
              max_angular=5,
              gaussian_sigma_constant=0.5,
              gaussian_sigma_type="Constant",
              cutoff_function_type="RadialScaling",
              cutoff_smooth_width=0.5,
              cutoff_function_parameters=
                    dict(
                            rate=1,
                            scale=3.5,
                            exponent=4
                        ),
              radial_basis="GTO",
              normalize=True,
            #   optimization=
            #         dict(
            #                 Spline=dict(
            #                    accuracy=1.0e-05
            #                 )
            #             ),
              compute_gradients=True
              )


soap = SphericalInvariants(**hypers)

managers = soap.transform(frame_test)

soap_array = managers.get_features(soap)
soap_grad_array = managers.get_features_gradient(soap)
print(soap_array.shape)
print(soap_grad_array.shape)

In [ ]:
from ase.visualize import view
view(frame_test)

In [ ]:
managers.get_representation_info()[1]

In [ ]:
managers.get_gradients_info()[0:10]

In [ ]:
# import numpy as np
soap_grad_array[20_000,853]

In [ ]:
soap.get_num_coefficients(5)
# soap.get_keys('C')

In [ ]:
rascal.neighbourlist.structure_manager.AtomsList(frame_test)

In [ ]:
managers.get_gradients_info().shape
# managers.get_representation_info()


In [ ]:
# https://github.com/lab-cosmo/librascal/blob/4e576ae7b9d3740715ab1910def5e1a15ffd1268/tests/python/python_representation_calculator_test.py
# n_neigh=2
# https://github.com/lab-cosmo/librascal/pull/317
n_feat=2250
ij = managers.get_gradients_info()
n_atoms = len(np.unique(ij[:, 1]))
n_neigh = ij.shape[0]

X_der = managers.get_features_gradient(soap).reshape((n_neigh, 3, n_feat))

In [ ]:
ij.shape

In [ ]:
X_der.shape

In [ ]:
ij.shape

In [ ]:
print(ij[:,1])

In [ ]:
X_der.shape

In [ ]:
import rascal as ras

sm = ras.load_structure("/media/dlb/la_cie1/repos/librascal/examples/i-PI/zundel/zundel_dataset.xyz")
sm = ras.compute_pairs(sm)
sm = ras.adaptor_strict(sm, 3.0)

rep = ras.SphericalInvariants(sm, hypers)
sm = rep.compute()

In [ ]:
import os
import ipyparallel as ipp

cluster = ipp.Cluster(n=4)
with cluster as rc:
    ar = rc[:].apply_async(os.getpid)
    pid_map = ar.get_dict()

In [ ]:
import ipyparallel as ipp
rc = ipp.Cluster(n=4).start_and_connect_sync()

In [ ]:
cluster = ipp.Cluster.from_file()

In [ ]:
ipp.__version__

In [ ]:
from ase.visualize import view

# view(frames)
print(frames[0].get_cell())

In [ ]:
 %matplotlib inline
from matplotlib import pylab as plt

import ase
from ase.io import read, write
from ase.build import make_supercell
from ase.visualize import view
import numpy as np
# If installed -- not essential, though
try:
    from tqdm.notebook import tqdm
except ImportError:
    tqdm = (lambda i, **kwargs: i)

from time import time

import rascal
from rascal.models import Kernel, train_gap_model, compute_KNM
from rascal.representations import SphericalInvariants
from rascal.utils import from_dict, to_dict, CURFilter, dump_obj, load_obj


## Run simulations